In [1]:
import numpy as np
import ipyvolume as ipv
from forest3d.geometry import Tree
from scipy.spatial import cKDTree
from scipy.interpolate import RegularGridInterpolator

In [2]:
tree_x, tree_y, tree_z = Tree(species='Douglas-fir', 
                              dbh=15, top_height=150, 
                              stem_x=0, stem_y=0, stem_z=0,
                              crown_shapes=((2.0, 1.5, 1.7, 2.3), (1.3, 1.2, 1.0, 1.1)),
                              crown_radii=(5,6,8,7)
                             ).get_crown()
tree_xs, tree_ys, tree_zs = tree_x.reshape((50,32)), tree_y.reshape((50,32)), tree_z.reshape((50,32))

In [26]:
grid_x = np.linspace(tree_x.min()-200, tree_x.max()+200,400)
grid_y = np.linspace(tree_y.min()-150, tree_y.max()+150,300)
grid_z = np.linspace(-10, tree_z.max()+10,100)
grid_xx, grid_yy, grid_zz = np.meshgrid(grid_x,grid_y,grid_z)
grid_xs = grid_xx.flatten()
grid_ys = grid_yy.flatten()
grid_zs = grid_zz.flatten()

In [16]:
ipv.figure(width=800)
ipv.scatter(grid_xs, grid_ys, grid_zs, marker="sphere", size=0.75, color='black')
# ipv.plot_surface(tree_xs, tree_ys, tree_zs, color="green")
ipv.plot_wireframe(tree_xs, tree_ys, tree_zs, color="green")
ipv.style.use('minimal')
ipv.show()

In [27]:
kdtree = cKDTree(data = np.stack((tree_x, tree_y, tree_z)).T)

In [28]:
grid_points = np.stack((grid_xs, grid_ys, grid_zs)).T

In [29]:
grid_distances = kdtree.query(grid_points)

In [30]:
interpolator = RegularGridInterpolator(points = (grid_x,grid_y,grid_z), 
                                       values = grid_distances[0].reshape(grid_x.shape[0], grid_y.shape[0], grid_z.shape[0]))

In [31]:
rand_x = (grid_x.max() - grid_x.min())*np.random.rand(50*32*5000) + grid_x.min()
rand_y = (grid_y.max() - grid_y.min())*np.random.rand(50*32*5000) + grid_y.min()
rand_z = (grid_z.max() - grid_z.min())*np.random.rand(50*32*5000) + grid_z.min()

In [20]:
ipv.scatter(rand_x, rand_y, rand_z, marker='sphere', color='blue', size=4)

Scatter(color=array('blue', dtype='<U4'), color_selected=array('white', dtype='<U5'), geo='sphere', line_mater…

In [34]:
%%time
distances = interpolator((rand_x, rand_y, rand_z))
distance_metric = distances.sum()
print(distance_metric)

1106924653.5756052
CPU times: user 20.8 s, sys: 7.53 s, total: 28.3 s
Wall time: 10.4 s


In [33]:
distances

array([199.24775936, 164.47346178, 152.46416422, ..., 118.77597072,
       163.45049417,  63.52945546])